# Conversation Data Isolation Sample Notebook

This notebook showcases the usage of data sources in MaximumGPT by using a local vector store with a single index and filter tags on different data sources. It showcases how data sources inherit filter tags when being created using assistants and conversations within that assistant, allowing data access to data ingested on assistant + data ingested on conversation level. 

# Network & Imports

In [ ]:
import json
import requests

from tests.api_assistant_creation import data_source_id

base_path= 'http://127.0.0.1:5002/api/v1'

# Use this if you are testing against an instance with OPENID auth
# %run fetch_openid_bearer.ipynb
# print (auth_headers)

def stream_response(r):
    r.encoding = 'utf-8'
    if r.status_code == 200:
        for line in r.iter_lines(): 
            json_data = json.loads(line.decode('utf-8'))
            if json_data['type'] == 'chunk':
                print(json_data['content'], end='')   
        
        print()   
    else: 
        print(r.text)
        
def stream_ingest(r):
    r.encoding = 'utf-8'
    if r.status_code == 200:
        for line in r.iter_lines(): 
            json_data = json.loads(line.decode('utf-8'))
            print (f"{json_data['currentStep']} of  {json_data['steps']}")
    else: 
        print(r.text)

# Setup MaximumGPT Modules

In [ ]:
# LLM Module GPT
r = requests.post(f'{base_path}/module', json=
{
  "name": "GPT35",
  "specId": "d7975f09-2b33-4ddf-bc29-cdc6c64a6781",
  "parameters": [
    {
      "name": "M_LLM_AZURE_OPENAI_API_KEY",
      "value": "*********"
    },{
      "name": "M_LLM_AZURE_OPENAI_API_VERSION",
      "value": "2023-07-01-preview"
    },{
      "name": "M_LLM_AZURE_OPENAI_DEPLOYMENT_NAME",
      "value": "aiphuahhailang01"
    },{
      "name": "M_LLM_AZURE_OPENAI_ENDPOINT",
      "value": "https://aiphuahhaiopai01.openai.azure.com"
    },{
      "name": "M_LLM_AZURE_OPENAI_MODEL_NAME",
      "value": "gpt-35-turbo"
    }
  ]
})

llm_id_gpt35 = r.json().get("id")
print("GPT35 ID: " + llm_id_gpt35)


# EMBEDDING_MODEL Module
r = requests.post(f'{base_path}/module', json=
{
  "name": "E",
  "specId": "846232c8-b6e9-4ee2-87ab-3e173768f06c",
  "parameters": [
    {
      "name": "M_EMB_AZURE_OPENAI_API_KEY",
      "value": "*********"
    },{
      "name": "M_EMB_AZURE_OPENAI_API_VERSION",
      "value": "2023-07-01-preview"
    },{
      "name": "M_EMB_AZURE_OPENAI_DEPLOYMENT_NAME",
      "value": "aiphuahhaiembb01"
    },{
      "name": "M_EMB_AZURE_OPENAI_ENDPOINT",
      "value": "https://aiphuahhaiopai01.openai.azure.com"
    },{
      "name": "M_EMB_AZURE_OPENAI_MODEL_NAME",
      "value": "text-embedding-ada-002"
    }
  ]
})

embedding_id = r.json().get("id")
print("Embedding model ID: " + embedding_id)

# VECTOR_STORE Module
r = requests.post(f'{base_path}/module', json=
{
  "name": "VS",
  "specId": "2b929513-16fa-47a4-94b5-7036e16fa8dc",
  "parameters": [{
    "name": "M_VECTOR_STORE_LOCAL_INDEX_NAME",
    "value": "data_isolation_idx"
  }]
})

vector_store_index_id = r.json().get("id")
print("Vector Store Index ID: " + vector_store_index_id)

# FILE_STORAGE Module
r = requests.post(f'{base_path}/module', json=
{
  "name": "FS",
  "specId": "b3a6b442-8123-49ee-b379-9ee1ff9099ae",
  "parameters": []
})

file_storage_id = r.json().get("id")
print("File Storage ID: " + file_storage_id)

# Setup Assistants

## System Instruction

In [ ]:
# First, we create a system instruction to push the model to restrict responses to the context knowledge only
r = requests.post(f'{base_path}/system-instruction', json={
  "name": "Enforce assistant knowledge",
  "message": """
    You must only respond using knowledge in the context of the user's request. Do try to answer the user prompt using other information than the provided context.
    """
})
system_instruction_id = r.json()['id']
print(f"System Instruction ID: {system_instruction_id}")

## Assistants
We are creating the assistants first without data source because we want to use the assistant id as filter-tag of a data source, so we create the data source after the assistants and then link them to the assistants.

In [ ]:
r = requests.post(f'{base_path}/assistant', json={
  "name": "A1",
  "description": "An Assistant with its own knowledge",
  "systemInstructionId": system_instruction_id,
  "customSystemInstruction": "At the end of each response, always append this text: 'Proudly presented by Assistant A1.'",
  "llmIds": [
    llm_id_gpt35
  ],
  "dataSourceIds": [],
  "public": False
})
assistant_1_id = r.json()['id']
print(f"Assistant A1 ID: {assistant_1_id}")

r = requests.post(f'{base_path}/assistant', json={
  "name": "A2",
  "description": "An Assistant with its own knowledge",
  "systemInstructionId": system_instruction_id,
  "customSystemInstruction": "At the end of each response, always append this text: 'Proudly presented by assistant A2.'",
  "llmIds": [
    llm_id_gpt35
  ],
  "dataSourceIds": [],
  "public": False
})
assistant_2_id = r.json()['id']
print(f"Assistant A2 ID: {assistant_2_id}")

## Data Sources
We are creating 2 data sources for the 2 different assistants now

In [ ]:
r = requests.post(f'{base_path}/assistant/{assistant_1_id}/data-source', json={
  "name": "DS-A1",
  "embeddingModelId": embedding_id,
  "vectorStoreId": vector_store_index_id,
  "fileStorageId": file_storage_id
})
data_source_a1_id = r.json()['id']
print(f"Data Source ID (Tagged for A1): {data_source_a1_id}")

r = requests.post(f'{base_path}/assistant/{assistant_2_id}/data-source', json={
  "name": "DS-A2",
  "embeddingModelId": embedding_id,
  "vectorStoreId": vector_store_index_id,
  "fileStorageId": file_storage_id
})
data_source_a2_id = r.json()['id']
print(f"Data Source ID (Tagged for A2): {data_source_a2_id}")

## Ingest assistant-specific data
We feed each data source with a particular knowledge now.

In [ ]:
# Feed knowledge of A1
data={
    'llmId': llm_id_gpt35,
    'dataSourceId': data_source_a1_id,
    'content': "One of the secrets of Mr. DataMcDaFace is: A1",
    'overrides': []
}

stream_ingest(requests.post(f'{base_path}/ingest/text', json=data))


# Feed knowledge of A2
data={
    'llmId': llm_id_gpt35,
    'dataSourceId': data_source_a2_id,
    'content': "One of the secrets of Mr. DataMcDaFace is: A2",
    'overrides': []
}

stream_ingest(requests.post(f'{base_path}/ingest/text', json=data))

# WHERE ARE WE NOW?

At this point, we have 2 assistant which 2 data sources, 1 dedicated for data of assistant A1 and one for A2. If you start chatting now with this data sources, your chat will already see content isolated for the according assistant.

However, we want to make it more fine granular now and introduce another later of knowledge by creating data sources with the context of a single conversation inside an assistant. To do so, we are following a similar approach like for the assistants (disclaimer: this behavior might be changed in the future!). 

## Creating Conversations

Two within one assistant

In [ ]:
r = requests.post(f'{base_path}/assistant/{assistant_1_id}/conversation', json={})
conversation_a1_1_id = r.json().get("id")
print("Conversation 1 for Assistant A1 ID: " + conversation_a1_1_id)

r = requests.post(f'{base_path}/assistant/{assistant_1_id}/conversation', json={})
conversation_a1_2_id = r.json().get("id")
print("Conversation 2 for Assistant A1 ID: " + conversation_a1_2_id)

print("*** Testing the knowledge so far.")

print("*** A1 - 1: What are the secrets of Mr. DataMcDaFace?")
r = requests.post(f'{base_path}/conversation/{conversation_a1_1_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "What are the secrets of Mr. DataMcDaFace?"
    }
  ],
  "overrides": []
})
stream_response(r)

print("*** A1 - 2: What are the secrets of Mr. DataMcDaFace?")
r = requests.post(f'{base_path}/conversation/{conversation_a1_2_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "What are the secrets of Mr. DataMcDaFace?"
    }
  ],
  "overrides": []
})
stream_response(r)

In [ ]:
r = requests.post(f'{base_path}/conversation/{conversation_a1_1_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "What are the secrets of Mr. DataMcDaFace?"
    }
  ],
  "overrides": []
})
stream_response(r)

## Creating conversation-specific Data Sources
For one of the conversations, create an isolated data source

In [ ]:
r = requests.post(f'{base_path}/conversation/{conversation_a1_1_id}/data-source', json={
  "name": "DS-A1-C1",
  "embeddingModelId": embedding_id,
  "vectorStoreId": vector_store_index_id,
  "fileStorageId": file_storage_id
})
data_source_a1_1_id = r.json()['id']
print(f"Data Source ID (Tagged for A1 - Conversation 1): {data_source_a1_1_id}")

## Ingest conversation specific data

In [ ]:
# Feed knowledge of A1
data={
    'llmId': llm_id_gpt35,
    'dataSourceId': data_source_a1_1_id,
    'content': "Another secret of Mr. DataMcDaFace is: A1C1",
    'overrides': []
}

r = requests.post(f'{base_path}/ingest/text', json=data)
print(str(r.content))

## Now Test if both conversations have know different things

In [ ]:
print("**** Setup *****")
assistant_json = requests.get(f'{base_path}/assistant/{assistant_1_id}').json()
assistant_ds_json = requests.get(f'{base_path}/data-source/{assistant_json["dataSourceIds"][0]}').json()
c1_json = requests.get(f'{base_path}/conversation/{conversation_a1_1_id}').json()
c1_ds_json = requests.get(f'{base_path}/data-source/{c1_json["dataSourceIds"][0]}').json()
c2_json = requests.get(f'{base_path}/conversation/{conversation_a1_2_id}').json()

print (f"Assistant '{assistant_json['name']}' --> Datasource '{assistant_ds_json['name']}'")
print (f"\tConversation '{c1_json['name']}' --> Datasource '{c1_ds_json['name']}'")
print (f"\tConversation '{c2_json['name']}' --> Datasource {c2_json['dataSourceIds']}'")

print("****************")

In [ ]:
print("*** A1 - 1: What are the secrets of Mr. DataMcDaFace?")
r = requests.post(f'{base_path}/conversation/{conversation_a1_1_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "What are the secrets of Mr. DataMcDaFace?"
    }
  ],
  "overrides": []
})
stream_response(r)

print("*** A1 - 2: What are the secrets of Mr. DataMcDaFace?")
r = requests.post(f'{base_path}/conversation/{conversation_a1_2_id}/message', json=
{
  "parts": [
    {
      "type": "TEXT",
      "content": "What are the secrets of Mr. DataMcDaFace?"
    }
  ],
  "overrides": []
})
stream_response(r)